In [294]:
import pandas as pd
import glob
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

basePath = r'D:\GitHub\crypto-jobs-scraper'
getroJobsPath = os.path.join(basePath, 'node-scraper', 'data', 'getro')
getroAggregatePath = os.path.join(basePath, 'data', 'getro', 'all_jobs.csv')
considerJobsPath = os.path.join(basePath, 'node-scraper', 'data', 'consider')
considerAggregatePath = os.path.join(basePath, 'data', 'consider', 'all_jobs.csv')
linkedinJobsPath = os.path.join(basePath, 'node-scraper', 'data', 'linkedin')
web3CareerJobsPath = os.path.join(basePath, 'data', 'web3_careers', 'page_data')
cryptoJobsListPath = os.path.join(basePath, 'data', 'crypto_jobs_list', 'all_jobs.csv')
crunchbaseDataPath = 'crunchbase_data.csv'

def csvFolderToDataframe(path):
    all_files = glob.glob(os.path.join(path , "*.csv"))
    
    li = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1

In [295]:
def processGetro():
    frame = csvFolderToDataframe(getroJobsPath)
    deduplicated = (frame.drop_duplicates(subset='Job Link', keep='first', inplace=False)
                    .drop_duplicates(subset='Getro ObjectID', keep='first', inplace=False))

    header = ["Company Name", "Job Link", "Job Location", "Job Title", "Salary Range", "Tags", "Posted Before"]
    deduplicated.to_csv(getroAggregatePath, columns = header, index=False)
    return deduplicated
    
getro = processGetro()

In [296]:
def processConsider():
    frame = csvFolderToDataframe(considerJobsPath)
    deduplicated = (frame.drop_duplicates(subset='Job Link', keep='first', inplace=False)
                    .drop_duplicates(subset='Consider JobID', keep='first', inplace=False))
    header = ["Company Name", "Job Link", "Job Location", "Job Title", "Salary Range", "Tags", "Posted Before"]
    deduplicated.to_csv(considerAggregatePath, columns = header, index=False)
    return deduplicated
    
consider = processConsider()

In [297]:
linkedIn = csvFolderToDataframe(path = linkedinJobsPath)
web3Career = csvFolderToDataframe(path = web3CareerJobsPath)
cryptoJobsList = pd.read_csv(cryptoJobsListPath, index_col=None, header=0)
crunchbaseData = pd.read_csv(crunchbaseDataPath, index_col=None, header=0)

In [298]:
print(getro.shape, consider.shape, linkedIn.shape, web3Career.shape, cryptoJobsList.shape, crunchbaseData.shape)

(7470, 10) (1584, 10) (72, 7) (17458, 7) (268, 4) (1000, 4)


In [299]:
# getroAndConsiderJobs = pd.concat([getro, consider])
# header = ["Company Name", "Job Link", "Job Location", "Job Title", "Salary Range", "Tags", "Posted Before"]
# vcBoardJobs = getroAndConsiderJobs.drop_duplicates(subset='Job Link', keep='first', inplace=False)[header]
# vcBoardJobs.to_csv(r'D:\GitHub\crypto-jobs-scraper\data\vc-job-boards\all_jobs.csv', columns = header, index=False)

In [300]:
concatenatedJobs = pd.concat([vcBoardJobs, linkedIn, web3Career, cryptoJobsList])
allJobs = concatenatedJobs.drop_duplicates(subset='Job Link', keep='first', inplace=False)
print(concatenatedJobs.shape, allJobs.shape)

(26234, 9) (25967, 9)


In [301]:
jobsPerCompany = allJobs.groupby(['Company Name'])[['Company Name']].size().reset_index(name='count').sort_values(by='count', ascending=False)

In [302]:
# finalDf = fuzzy_merge(crunchbaseData.head(100), jobsPerCompany, 'Company Name', 'Company Name', threshold=80, limit=1)

In [303]:
# joined = finalDf.merge(jobsPerCompany, left_on='matches', right_on='Company Name', how='left').fillna('')

In [304]:
# joined[joined['count']!=''].shape

In [305]:
# joined.to_csv('matched_data.csv', index=False)

In [306]:
def getTopNJoinResult(crunchbase, jobsPerCompany, n, key = 'Company Name'):
    crunchbaseTopN = crunchbase.head(n)
    result = crunchbaseTopN.merge(jobsPerCompany, left_on=key, right_on=key, how='left').fillna('')
    return result

def printCoverage(result, n,key = 'Company Name'):
    scraped = result[result['count']!='']
    shape = scraped.shape
    uniqueCompaniesCount = scraped[key].nunique()
    print(round(uniqueCompaniesCount/n*100), "% match", "for Top", n)

banned = ['foundation', 'industries', 'protocol', 'foundation', 'inc.', 'llc', 'labs', 'limited', 'chain', 'inc', 'network', 'group']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
    
crunchbaseCleaned = crunchbaseData.copy()
crunchbaseCleaned['Company Name Lowercase'] = crunchbaseCleaned['Company Name'].str.lower()
crunchbaseCleaned['Company Name Cleaned'] = crunchbaseCleaned['Company Name Lowercase'].apply(f)

jobsPerCompanyCleaned = jobsPerCompany.copy()
jobsPerCompanyCleaned['Company Name Lowercase'] = jobsPerCompanyCleaned['Company Name'].str.lower()
jobsPerCompanyCleaned['Company Name Cleaned'] = jobsPerCompanyCleaned['Company Name Lowercase'].apply(f)

steps = [100,250,500,1000]

print("no modifications")
for step in steps:
    result = getTopNJoinResult(crunchbaseData, jobsPerCompany, step)
    printCoverage(result, step)

print("lowercase + banning generic keywords")
for step in steps:
    result = getTopNJoinResult(crunchbaseCleaned, jobsPerCompanyCleaned, step, 'Company Name Cleaned')
    printCoverage(result, step, 'Company Name Cleaned')

no modifications
58 % match for Top 100
51 % match for Top 250
44 % match for Top 500
32 % match for Top 1000
lowercase + banning generic keywords
65 % match for Top 100
57 % match for Top 250
48 % match for Top 500
36 % match for Top 1000


In [307]:
result = getTopNJoinResult(crunchbaseCleaned, jobsPerCompanyCleaned, 1000, 'Company Name Cleaned')
result.to_csv('top_1000_companies.csv', index=False)

In [308]:
# print(pd.Series(' '.join(crunchbaseCleaned['Company Name Cleaned']).lower().split()).value_counts().to_string())
# jobsPerCompanyCleaned[jobsPerCompanyCleaned.duplicated(['Company Name Cleaned'], keep=False)].sort_values("Company Name Cleaned")